In [ ]:
import torch
from mobilenet import MobileNetSmall3D
from torchvision.io import read_video
!pip install -q mediapy
import mediapy
!pip install -q av
#@title Run this cell to run Picklebot for your choice of video

def classify_pitch(confidence_scores):
    if torch.argmax(confidence_scores) == 0:
        call = 'ball'
    elif torch.argmax(confidence_scores) == 1:
        call = 'strike'
    else:
        print("that's odd, something is wrong")
        pass
    return call


ball_video = 'demo_files/clip_7765.mp4'
strike_video = 'demo_files/clip_53102.mp4'
wild_card = 'demo_files/wildcard.mp4'

pitch_choice = 'Ball' #@param ['Ball', 'Strike', 'Wild Card']

choice_map = {
    'Ball':ball_video,
    'Strike':strike_video,
    'Wild Card':wild_card
}

pitch = choice_map[pitch_choice]

video = mediapy.read_video(pitch)
mediapy.show_video(video)

pitch_tensor = (read_video(pitch,pts_unit='sec')[0].permute(-1,0,1,2)).unsqueeze(0)/255

model = MobileNetSmall3D()
model.load_state_dict(torch.load('models/mobilenet_small.pth'))
#load the dictionary
model.eval()
output = model(pitch_tensor)
call = classify_pitch(output)

print(f"This pitch was a {call}!")